Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called predict_message that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the SMS Spam Collection dataset. The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells

In [21]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import re
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer

print(tf.__version__)

You should consider upgrading via the 'c:\users\mail2\envs\mlearn\scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\mail2\envs\mlearn\scripts\python.exe -m pip install --upgrade pip' command.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mail2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


2.6.0-dev20210407


In [1]:
import pandas as pd
import re
import numpy as np
import array as arr
#import tensorflow as tf
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras
import nltk
#nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
# get data files
#!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
#!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "C:/Users/mail2/Envs/mlearn/train-data.tsv"
test_file_path = "C:/Users/mail2/Envs/mlearn/valid-data.tsv"

In [4]:
training_dataset = pd.read_csv('C:/Users/mail2/Envs/mlearn/train-data.tsv', sep='\t',header=None,names=['class','text'])
testing_dataset = pd.read_csv('C:/Users/mail2/Envs/mlearn/valid-data.tsv', sep='\t',header=None,names=['class','text'])

In [5]:
training_dataset.head()

,class,text
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [6]:
len(testing_dataset)

1392

In [7]:
#clean the data
def clean_the_dataset(dataset):
    for j in range(len(dataset)):
        sen = dataset.iloc[j,1]
        punc = '''()-[]{}:;'"\,<>./?#$%^&*_~'''
        #print(sen)
        for ele in sen:
            if ele in punc:
                sen=sen.replace(ele,'')
      
        w=re.split(r"\s",sen)

    #print(w)
    #sentence=[]
        for i in range(len(w)):
            if(len(w[i])==1 or len(w[i])==2):
                w[i]=""
            if(w[i].isnumeric()):
                w[i]='number'
            w[i]=re.sub(r"[\w\W]+[\d]+[\w]+",'alpha_number', w[i])
            w[i]=re.sub(r"(www\w+)",'web_site', w[i])
     
        while("" in w) :
            w.remove("")
        sentence = " ".join(w)    
        dataset.iat[j,1]=sentence
    return dataset

In [8]:
training_dataset = clean_the_dataset(training_dataset)
testing_dataset = clean_the_dataset(testing_dataset)

In [9]:
training_dataset.shape

(4179, 2)

In [10]:
def prepare_the_data(dataset):
    words = []
    classes = []
    documents = []
    stemmer = LancasterStemmer()
    ignore_words = ['?','!','#', '&', "'ll", "'m", "'s", ')', '*', ',', '-', '.', '..', '...',':', ';','',]
    # loop through each sentence in our training data
    for pattern in dataset.to_numpy():
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern[1])
        # add to our words list

        words.extend(w)
        # add to documents in our corpus
        documents.append((w, pattern[0]))
        # add to our classes list
        if pattern[0] not in classes:
            classes.append(pattern[0])

    # stem and lower each word and remove duplicates
    words = [stemmer.stem(word.lower()) for word in words if word not in ignore_words]
    uniqwords = list(set(words))

    # remove duplicates
    classes = list(set(classes))
    return words,classes,documents
    
    

In [11]:
    train_stemmedwords,classes, training_doc = prepare_the_data(training_dataset)
    print (len(training_doc), "training data documents")
    print (len(classes), "classes", classes)
    print (len(train_stemmedwords), "train_stemmedwords") #train_stemmedwords

    test_stemmedwords,classes, testing_doc = prepare_the_data(testing_dataset)
    print (len(testing_doc), "training data documents")
    print (len(classes), "classes", classes)
    print (len(test_stemmedwords), "test_stemmedwords") #, test_stemmedwords

4179 training data documents
2 classes ['spam', 'ham']
48140 train_stemmedwords
1392 training data documents
2 classes ['spam', 'ham']
15962 test_stemmedwords


In [134]:
# finding most used words

words_counted=np.unique(train_stemmedwords, return_counts=True)
vocab_list=[]
# df_books1=df_books.copy()
len(words_counted[0])
for i in range(len(words_counted[0])):
    if words_counted[1][i]>=75:
    #df_books1.drop(df_books1[df_books1.isbn== books_rated[0][i]].index, inplace = True)
        vocab_list.append(words_counted[0][i])
len(vocab_list)


114

In [135]:
vocab_list

['about',
 'aft',
 'al',
 'alpha_number',
 'and',
 'any',
 'ar',
 'ask',
 'back',
 'been',
 'but',
 'cal',
 'can',
 'cant',
 'car',
 'com',
 'day',
 'dear',
 'did',
 'dont',
 'ev',
 'fin',
 'for',
 'fre',
 'friend',
 'from',
 'get',
 'giv',
 'going',
 'good',
 'got',
 'gre',
 'happy',
 'has',
 'hav',
 'her',
 'hey',
 'him',
 'hom',
 'hop',
 'how',
 'il',
 'it',
 'just',
 'know',
 'lat',
 'lik',
 'lor',
 'lov',
 'ltgt',
 'mak',
 'meet',
 'mess',
 'min',
 'miss',
 'mobl',
 'mor',
 'much',
 'nee',
 'new',
 'night',
 'not',
 'now',
 'numb',
 'off',
 'on',
 'our',
 'out',
 'phon',
 'pleas',
 'pls',
 'real',
 'reply',
 'say',
 'see',
 'send',
 'she',
 'som',
 'sorry',
 'stil',
 'stop',
 'tak',
 'tel',
 'text',
 'that',
 'the',
 'then',
 'ther',
 'they',
 'thi',
 'thing',
 'think',
 'tim',
 'today',
 'tomorrow',
 'too',
 'txt',
 'wait',
 'want',
 'was',
 'wat',
 'way',
 'week',
 'wel',
 'what',
 'when',
 'wher',
 'who',
 'wil',
 'win',
 'with',
 'work',
 'yo',
 'you']

In [136]:
def prepare_bag(documents):
    Xdata = []
    ydata = []
    stemmer = LancasterStemmer()
   
#     df = pd.DataFrame()
    
    # training set, bag of words for each sentence
    for doc in documents:
        # initialize our bag of words
        bag=[]
        # list of tokenized words for the pattern
        pattern_words = doc[0]
        # stem each word
        pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
        # create our bag of words array
        for w in vocab_list:
            bag.append(1) if w in pattern_words else bag.append(0)

        Xdata.append(bag)
        # output is a '0' for each tag and '1' for current tag
        #output_row = list(output_empty)
        #output_row[classes.index(doc[1])] = 1
        y = 1 if doc[1]=="spam" else 0
        ydata.append(y)
#        data ={'x':bag,'y':y}
#     df= df.append(data,ignore_index=True)
    return Xdata,ydata


In [137]:
training_data,training_labels=prepare_bag(training_doc)
testing_data,testing_labels=prepare_bag(testing_doc)
# target=training_df.pop('y')

# training_dataset_tf = tf.data.Dataset.from_tensor_slices((training_df.values, target.values))

In [138]:
training_doc[0]
len(training_data[0])

114

In [142]:
# from tensorflow.keras import layers
# model = tf.keras.Sequential([
  
  
#  # layers.Dense(265),
#   #layers.Dropout(0.2),
#   layers.Dense(1)])
model = keras.Sequential([
#     keras.layers.Input(shape=(182,1)),
    keras.layers.Dense(units=256,activation='relu',input_shape=(114,)),
    keras.layers.Dense(units=128,activation='relu'),
    keras.layers.Dense(units=1)
]) 
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 256)               29440     
_________________________________________________________________
dense_10 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 129       
Total params: 62,465
Trainable params: 62,465
Non-trainable params: 0
_________________________________________________________________


In [143]:
model.compile(optimizer ='adam',
                 loss ='binary_crossentropy',
                 metrics =['accuracy'])

In [144]:
history=model.fit(training_data, training_labels,
                    epochs=16
                    )

Epoch 1/16
131/131 [==============================] - 2s 2ms/step - loss: 0.2573 - accuracy: 0.9513
Epoch 2/16
131/131 [==============================] - 0s 3ms/step - loss: 0.1374 - accuracy: 0.9798
Epoch 3/16
131/131 [==============================] - 1s 3ms/step - loss: 0.1071 - accuracy: 0.9902
Epoch 4/16
131/131 [==============================] - 1s 3ms/step - loss: 0.1193 - accuracy: 0.9911
Epoch 5/16
131/131 [==============================] - 1s 4ms/step - loss: 0.0844 - accuracy: 0.9934
Epoch 6/16
131/131 [==============================] - 0s 3ms/step - loss: 0.0877 - accuracy: 0.9936
Epoch 7/16
131/131 [==============================] - 1s 4ms/step - loss: 0.0909 - accuracy: 0.9934: 0s - loss: 0.0909 - accuracy: 0.99
Epoch 8/16
131/131 [==============================] - 0s 3ms/step - loss: 0.1006 - accuracy: 0.9932
Epoch 9/16
131/131 [==============================] - 1s 5ms/step - loss: 0.0836 - accuracy: 0.9938
Epoch 10/16
131/131 [==============================] - 0s 3ms/st

In [46]:
loss, accuracy = model.evaluate(testing_data,testing_labels)

44/44 [==============================] - 0s 1ms/step - loss: 0.1264 - accuracy: 0.9849


In [51]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = []
    for w in words:
            bag.append(1) if w in sentence_words else bag.append(0)
    return(bag)

def clean_up_sentence(sentence):
    # tokenize the pattern
    stemmer = LancasterStemmer()
    ignore_words = ['?','!','#', '&', "'ll", "'m", "'s", ')', '*', ',', '-', '.', '..', '...',':', ';','',]
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words if word not in ignore_words]
    return sentence_words

In [119]:
w= bow("i'm going to boston",vocab_list,True)
print([w])
w=[[-1.2300739]]
w[0][0]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


-1.2300739

In [127]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
   # pred_text1=[pred_text,]
#     Xpred=[]
#     for p in pred_text:
    bag=bow(pred_text,vocab_list,True)
#         Xpred.append(bag)
    probability = model.predict([bag])
    
    if probability[0][0] < 0:
         ans='ham'  
    else: 
        ans='spam'
    result=[probability[0][0], ans]
    return result

#pred_text = "how are you doing today?"
# predict_message(pred_text)
# print(prediction)

In [128]:
pred_text = "how are you doing today?"
prediction=predict_message(pred_text)
print(prediction)


[-1.2300739, 'ham']


In [129]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
